<a href="https://colab.research.google.com/github/AmiraOsamaMohamed/GraduationProject/blob/main/GProject_ipyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#preprocess

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
import re
import spacy
from nltk import word_tokenize , sent_tokenize
from nltk.corpus import stopwords

In [3]:
#read json file and convert it to DataFrame
import json
def read_data(path):
 data = []

 with open(path) as trainFile:
    for line in trainFile:
        data.append(json.loads(line))
 all_data=pd.DataFrame(data)
 all_data.head()
 #split col(answers) to text and start_char and combine them as base columns in big DataFrame
 line=[]
 list_text=[]
 list_start_char=[]
 list_pq_id=[]

 for i in range(0,len(data)):
   line.extend(data[i]['answers'])


 for i in range(0,len(data)):
   list_text.append(line[i]['text'])



 for i in range(0,len(data)):
   list_start_char.append(line[i]['start_char'])



 for i in range(0,len(data)):
   list_pq_id.append(data[i]['pq_id'])



 answer_df=pd.DataFrame(list_pq_id,columns=['pq_id'])
 answer_df['text']=list_text
 answer_df['start_char']=list_start_char
 answer_df.head()

#merge between 2 DataFrame
 merged_data = pd.merge(all_data, answer_df, on="pq_id")
 merged_data = merged_data.drop(columns=["answers"])
 merged_data = merged_data.rename(columns={"text":"answer"})
 merged_data.head()
 return merged_data

In [4]:
#convert merged data to csv file
train=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.jsonl')
validation=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.jsonl')
test=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_test_gold.jsonl')
train.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.csv', index=False)
validation.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.csv', index=False)
test.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_test_gold.csv', index=False)

In [5]:
#clean text
def clean(text):
    text = re.sub(r"http\S+", " ", text)  # remove urls
    text = re.sub(r"@[\w]*", " ", text)  # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r"\t", " ", text)  # remove tabs
    text = re.sub(r"\n", " ", text)  # remove line jump
    text = re.sub(r"\s+", " ", text)  # remove extra white space
    text = re.sub(r'[^\w\s]', '', text) # Removing punctuations in string using regex
    text = text.strip()
    return text

In [6]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# Function to tokenize sentences
def tokenize_words(text):
    return word_tokenize(text)

# Function to remove stopwords from tokenized text
stop_words = set(stopwords.words('arabic'))
def remove_stopwords(tokenized_text):
    return [word for word in tokenized_text if word not in stop_words]


# Function to return to the base of word(stemmer) from tokenized text
stemmer = SnowballStemmer('arabic')
def stem_words(tokenized_text):
    return [stemmer.stem(word) for word in tokenized_text]


def processed_text(text):
  cleaned_text=clean(text)
  tokenized_text=tokenize_words(cleaned_text)
  underProcessing_text=remove_stopwords(tokenized_text)
  processed_text=stem_words(underProcessing_text)
  return processed_text


# Apply the functions(processing) to the DataFrame
train['processed passage'] = train['passage'].apply(processed_text)


train['processed question'] = train['question'].apply(processed_text)


train['processed answer'] = train['answer'].apply(processed_text)



# Print the DataFrame
train.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,pq_id,passage,surah,verses,question,answer,start_char,processed passage,processed question,processed answer
0,2:1-5_372,الم. ذلك الكتاب لا ريب فيه هدى للمتقين. الذين ...,2,1-5,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,الذين يؤمنون بما أنزل إليك وما أنزل من قبلك,97,"[الم, كتاب, ريب, هدى, متق, يومن, غيب, يقيم, صل...","[دلايل, قرء, تاليف, سيد, محمد]","[يومن, انزل, انزل, قبل]"
1,2:23-24_372,وإن كنتم في ريب مما نزلنا على عبدنا فأتوا بسور...,2,23-24,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,إن كنتم في ريب مما نزلنا على عبدنا فأتوا بسورة...,1,"[كنتم, ريب, نزل, عبد, اتو, سور, مثل, وادع, شهد...","[دلايل, قرء, تاليف, سيد, محمد]","[كنتم, ريب, نزل, عبد, اتو, سور, مثل]"
2,2:40-48_372,يا بني إسرائيل اذكروا نعمتي التي أنعمت عليكم و...,2,40-48,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,آمنوا بما أنزلت مصدقا لما معكم,85,"[بن, اسراييل, اذكر, نعم, انعم, عل, اوف, عهد, ا...","[دلايل, قرء, تاليف, سيد, محمد]","[امنو, انزل, مصدق, معكم]"
3,2:91-93_372,وإذا قيل لهم آمنوا بما أنزل الله قالوا نؤمن بم...,2,91-93,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,إذا قيل لهم آمنوا بما أنزل الله,1,"[قيل, امنو, انزل, الله, قالو, نوم, انزل, عل, ي...","[دلايل, قرء, تاليف, سيد, محمد]","[قيل, امنو, انزل, الله]"
4,2:97-101_372,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,2,97-101,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,هو الحق مصدقا لما معهم,78,"[قل, عدو, جبريل, فان, نزل, قلب, باذ, الله, مصد...","[دلايل, قرء, تاليف, سيد, محمد]","[الحق, مصدق, معهم]"


In [7]:
validation['processed passage'] = validation['passage'].apply(processed_text)


validation['processed question'] = validation['question'].apply(processed_text)


validation['processed answer'] = validation['answer'].apply(processed_text)

# Print the DataFrame
validation.head()

,pq_id,passage,surah,verses,question,answer,start_char,processed passage,processed question,processed answer
0,13:18-24_360,للذين استجابوا لربهم الحسنى والذين لم يستجيبوا...,13,18-24,هل سيجمع الله بين المؤمنين وأبنائهم وأهلهم في ...,جنات عدن يدخلونها ومن صلح من آبائهم وأزواجهم و...,451,"[ذين, استجاب, لرب, حسنى, يستجيب, ارض, جميع, مث...","[سيجمع, الله, مومن, ابناء, اهل, الجن]","[جنا, عدن, يدخل, صلح, اباء, ازواج, وذر]"
1,40:7-9_360,الذين يحملون العرش ومن حوله يسبحون بحمد ربهم و...,40,7-9,هل سيجمع الله بين المؤمنين وأبنائهم وأهلهم في ...,ربنا وأدخلهم جنات عدن التي وعدتهم ومن صلح من آ...,156,"[يحمل, عرش, حول, يسبح, حمد, ربهم, يومن, يستغفر...","[سيجمع, الله, مومن, ابناء, اهل, الجن]","[ربن, ادخل, جنا, عدن, وعد, صلح, اباء, ازواج, وذر]"
2,52:17-28_360,إن المتقين في جنات ونعيم. فاكهين بما آتاهم ربه...,52,17-28,هل سيجمع الله بين المؤمنين وأبنائهم وأهلهم في ...,الذين آمنوا واتبعتهم ذريتهم بإيمان ألحقنا بهم ...,151,"[متق, جنا, نعيم, فاكه, اتا, ربهم, ووق, ربهم, ع...","[سيجمع, الله, مومن, ابناء, اهل, الجن]","[امنو, واتبع, ذر, ايم, الحق, ذر]"
3,73:15-19_386,إنا أرسلنا إليكم رسولا شاهدا عليكم كما أرسلنا ...,73,15-19,هل هناك علاقة بين الخوف أو الاضطرابات النفسية ...,كيف تتقون إن كفرتم يوما يجعل الولدان شيبا,102,"[ارسل, رسول, شاهد, عل, ارسل, رعو, رسول, عصى, ر...","[علاق, خوف, اضطراب, نفس, شيب]","[تتقو, فرتم, يوم, يجعل, ولد, شيب]"
4,28:76-81_114,إن قارون كان من قوم موسى فبغى عليهم وآتيناه من...,28,76-81,من الذي خسف الله به الأرض؟,قارون,479,"[قارو, قوم, موسى, بغى, عل, اتين, كنوز, مفاتح, ...","[خسف, الله, ارض]",[قارو]


In [8]:
test['processed passage'] = test['passage'].apply(processed_text)


test['processed question'] = test['question'].apply(processed_text)


test['processed answer'] = test['answer'].apply(processed_text)

test.head()

,pq_id,passage,surah,verses,question,answer,start_char,processed passage,processed question,processed answer
0,2:1-5_570,الم. ذلك الكتاب لا ريب فيه هدى للمتقين. الذين ...,2,1-5,من هو المؤمن بتعريف القرآن؟,الذين يؤمنون بالغيب ويقيمون الصلاة ومما رزقناه...,40,"[الم, كتاب, ريب, هدى, متق, يومن, غيب, يقيم, صل...","[موم, تعريف, قرء]","[يومن, غيب, يقيم, صلا, ومم, رزقن, ينفق]"
1,2:1-5_610,الم. ذلك الكتاب لا ريب فيه هدى للمتقين. الذين ...,2,1-5,ما الرد على الزعم باحتواء القرآن مفاهيم عقلية ...,الذين يؤمنون بما أنزل إليك وما أنزل من قبلك وب...,97,"[الم, كتاب, ريب, هدى, متق, يومن, غيب, يقيم, صل...","[الرد, زعم, احتواء, قرء, مفاهيم, عقل, علم, مغلوط]","[يومن, انزل, انزل, قبل, اخر, يوقن]"
2,2:21-22_505,يا أيها الناس اعبدوا ربكم الذي خلقكم والذين من...,2,21-22,ما الدلائل على أن القرآن أُنزل لجميع الناس أو ...,ذلك الكتاب لا ريب فيه,5,"[ناس, اعبد, ربكم, خلق, قبل, لعل, تتقو, ارض, را...","[دلايل, قرء, انزل, جميع, ناس, امم, شعوب]","[كتاب, ريب]"
3,2:34-39_568,وإذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس...,2,34-39,لماذا يعادي الشيطان الإنسان؟,يا أيها الناس اعبدوا ربكم الذي خلقكم والذين من...,0,"[قلن, ملايك, اسجد, لادم, سجد, ابليس, ابى, واست...","[لماذ, يعاد, شيط, انس]","[ناس, اعبد, ربكم, خلق, قبل]"
4,2:60-62_611,وإذ استسقى موسى لقومه فقلنا اضرب بعصاك الحجر ف...,2,60-62,هل هناك دليل أو أدلة على وحدة الأديان التي أنز...,إذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس ...,1,"[استسقى, موسى, لقوم, فقل, اضرب, بعص, حجر, فانف...","[دليل, ادل, وحد, الاد, انزل, الله]","[قلن, ملايك, اسجد, لادم, سجد, ابليس, ابى, واست..."


In [9]:
train.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.csv', index=False)
validation.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.csv', index=False)
test.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_test_gold.csv', index=False)

#transformer

In [10]:
! git clone https://gitlab.com/bigirqu/quranqa.git

Cloning into 'quranqa'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 333 (delta 43), reused 86 (delta 43), pack-reused 247
Receiving objects: 100% (333/333), 312.88 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [11]:
! pip install farasa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 51.5 MB/s eta 0:00:00


In [12]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [13]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.1 MB/s eta 0:00:00


In [16]:
!ls /content/drive/MyDrive/data

QQA23_TaskB_qrcd_v1.2_dev.csv	       QQA23_TaskB_qrcd_v1.2_test_noAnswers.jsonl  README.md
QQA23_TaskB_qrcd_v1.2_dev.jsonl        QQA23_TaskB_qrcd_v1.2_train.csv		   runs
QQA23_TaskB_qrcd_v1.2_test_gold.csv    QQA23_TaskB_qrcd_v1.2_train.jsonl
QQA23_TaskB_qrcd_v1.2_test_gold.jsonl  quranqa22_eval.cpython-310.pyc


In [18]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import numpy as np
import json
import sys
from tqdm import tqdm
import time
sys.path.insert(0,"/content/drive/MyDrive/data/quranqa22_eval.cpython-310.pyc")
import quranqa22_eval as eval #/content/quranqa/code/quranqa22_eval.py

In [19]:
train = pd.read_csv("/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.csv")
validation = pd.read_csv("/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.csv")

In [20]:
train_dataset_jsonl = []
with open('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

with open('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

In [21]:
len(train_dataset_jsonl)

1155

In [22]:
data = pd.concat([train, validation])

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155 entries, 0 to 162
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pq_id               1155 non-null   object
 1   passage             1155 non-null   object
 2   surah               1155 non-null   int64 
 3   verses              1155 non-null   object
 4   question            1155 non-null   object
 5   answer              1155 non-null   object
 6   start_char          1155 non-null   int64 
 7   processed passage   1155 non-null   object
 8   processed question  1155 non-null   object
 9   processed answer    1155 non-null   object
dtypes: int64(2), object(8)
memory usage: 99.3+ KB


In [24]:
data = data.filter(items=["pq_id","passage","question"])
data_unique_QA = data.drop_duplicates()

In [25]:
len(data_unique_QA)

1155

In [26]:
from transformers.utils.dummy_pt_objects import RemBertForSequenceClassification
def answer_question(row):
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result_row = pipe(question=row["question"], context=row["passage"], handle_impossible_answer=True, topk=5)
    return result_row

In [27]:
def Try_Transformer(tokenizer_name,model_name ):
  result = {}
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  # Apply the function to the "question" and "passage" columns of the DataFrame
  for i in tqdm(range(len(data_unique_QA))):
    result_part = answer_question(data_unique_QA.iloc[i])
    result.update({data_unique_QA.iloc[i]["pq_id"]:result_part})
  return result

In [28]:
all_scores = {}

*try transformer*

[CAMeL-Lab/bert-base-arabic-camelbert-mix](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix)

[CAMeL-Lab/bert-base-arabic-camelbert-ca](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-ca)

[bert-base-cased](https://huggingface.co/bert-base-cased)

[bert-base-uncased](https://huggingface.co/bert-base-uncased)

[aubmindlab/araelectra-base-generator](https://huggingface.co/aubmindlab/araelectra-base-generator)


[aubmindlab/araelectra-base-discriminator](https://huggingface.co/aubmindlab/araelectra-base-discriminator)

[aubmindlab/bert-large-arabertv2](https://huggingface.co/aubmindlab/bert-large-arabertv2)

In [31]:
tokenizer = "IProject-10/bert-base-uncased-finetuned-squad2"
model = "IProject-10/bert-base-uncased-finetuned-squad2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1155/1155 [58:26<00:00,  3.04s/it]


{'pRR': 0.05689177348514567, 'exact_match': 0.001221001221001221, 'f1': 0.00681045635270636}
